<a href="https://colab.research.google.com/github/ilsilfverskiold/transformers-nlp-docs/blob/main/cook/fine-tune/fine_tune_encoder_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install dependencies
!pip install -U datasets
!pip install -U accelerate
!pip install -U transformers
!pip install -U huggingface_hub

In [ ]:
# connect to drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

# import file - make sure you set the correct path in your Google Drive (this would be the file I'm importing)
# my file has two fields I'll be using called 'text' and 'keywords'
file_path = '/content/drive/My Drive/8500_rows_keywords.csv'
df = pd.read_csv(file_path)
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

# remove any null values for the 'keywords' field - will cause issues later if there are any
df = df[df['keywords'].notnull()]

# Split dataset into training and temp (15% - 7.5% each) (for validation and testing)
train_df, temp_df = train_test_split(df, test_size=0.15, random_state=42)

# Split temp into validation and testing
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [ ]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
# create a dataset dict with the train, validate and test set
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# print the dict
dataset_dict

In [ ]:
# (Optional) map out some examples from the dataset - my fields I'm using are text and keywords (make sure you set your own)
def show_samples(dataset, num_samples=10, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Text: {example['text']}'")
        print(f"'>> Keywords: {example['keywords']}'")


show_samples(dataset_dict)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# set the correct model you'll be fine-tuning (see seq-to-seq model docs for more information)
model_name = 'facebook/bart-large'
# get the tokenizer for the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# check the token length of the keywords field - you can do this for both fields
texts = dataset_dict['train']['keywords']

# Tokenize all texts and find the maximum length (max for BART is 1024 tokens)
max_token_length = max(len(tokenizer.encode(text, truncation=True)) for text in texts)
print(f"The longest text is {max_token_length} tokens long.")

In [ ]:
# convert both the input text and the target text into a format suitable for training a sequence-to-sequence model
# remember data preprocessing functions would look different if you were using a model with a different architecture, such as an encoder-only or decoder-only model.

def get_feature(batch):
  encodings = tokenizer(batch['text'], text_target=batch['keywords'],
                        max_length=1024, truncation=True)

  encodings = {'input_ids': encodings['input_ids'],
               'attention_mask': encodings['attention_mask'],
               'labels': encodings['labels']}

  return encodings

dataset_pt = dataset_dict.map(get_feature, batched=True)

In [ ]:
# the dataset should be formatted as PyTorch tensors with only the new fields
# i.e. specifies which columns should be returned when accessing the data - only the new fields will be returned
columns = ['input_ids', 'labels', 'attention_mask']
dataset_pt.set_format(type='torch', columns=columns)

dataset_pt

In [ ]:
# the data collator is responsible for dynamically padding the batches to the maximum length in each batch.
# which is crucial for efficient training of transformer models like BART or T5.
# padding will look different depending on the type of model you use, you can see here that this one is specifically for seq-to-seq

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from transformers import TrainingArguments, Trainer

# start training the model
# we're using the Trainer API which abstracts away a lot of complexity
training_args = TrainingArguments(
    output_dir = 'bart_tech_keywords',
    num_train_epochs=3, # your choice
    warmup_steps = 500,
    per_device_train_batch_size=4, # keep a small batch size when working with a small GPU
    per_device_eval_batch_size=4,
    weight_decay = 0.01, # helps prevent overfitting
    logging_steps = 10,
    evaluation_strategy = 'steps',
    eval_steps=50, # base this on the size of your dataset and number of training epochs
    save_steps=1e6,
    gradient_accumulation_steps=16 # running this on a small GPU
)

trainer = Trainer(model=model, args=training_args, tokenizer=tokenizer, data_collator=data_collator,
                  train_dataset = dataset_pt['train'], eval_dataset = dataset_pt['validation'])

trainer.train()

# note: the trainer loss should go down, the validation loss may fluctuate for each evaluation step but consistently increasing validation, while training is going down could be a sign of overfitting

In [ ]:
# save the model
trainer.save_model('tech-keywords-extractor')

In [ ]:
from transformers import pipeline

# test the model using Hugging Face's pipeline
pipe = pipeline('summarization', model='tech-keywords-extractor')

# test the first item in the test set to see how it does
test_text=dataset_dict['test'][0]['text']
keywords = dataset_dict['test'][0]['keywords']
print("the text: ", text_test)
print("generated keywords: ", pipe(test_text))
print("orginal keywords : ",keywords)

In [ ]:
# iterate over the test set to generate 50 examples at once
for i in range(0, 50):
    text_test = dataset_dict['test'][i]['text']
    keywords = dataset_dict['test'][i]['keywords']
    print("text: ", text_test)
    print("generated keywords: ", pipe(text_test)[0]['summary_text'])
    print("original keywords: ", keywords)

In [ ]:
# if you're satisfied we can push it to Hugging Face
# you'll need a token from your Hugging Face account to log in
!huggingface-cli login

In [ ]:
# you would replace your own name here
# you do not need to create a repository beforehand
trainer.push_to_hub("ilsilfverskiold/tech-keywords-extractor")